# Task 1: preRT segmentation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from monai.transforms import LoadImage
from monai.data import Dataset, DataLoader
from monai.losses import DiceLoss
from monai.metrics import DiceMetric
from monai.networks.nets import UNet
from monai.utils import set_determinism
import os.path
import random
import torch

from monai.transforms import (
    Compose,
    LoadImaged,
    Compose,
    LoadImaged,
    NormalizeIntensityd,
    Orientationd,
    Spacingd,
    EnsureTyped,
    EnsureChannelFirstd,
    ToTensord,
    Resized,
    AsDiscreted
)

In [ ]:
data_path = "/cluster/projects/vc/data/mic/open/HNTS-MRG/train/"

In [ ]:
data_preRT = []
for patient_num in os.listdir(data_path):
    patient = f"{data_path}{patient_num}"
    image = f"{patient}/preRT/{patient_num}_preRT_T2.nii.gz"
    mask = f"{patient}/preRT/{patient_num}_preRT_mask.nii.gz"
    
    data_preRT.append({"image": image, "label": mask})

print(len(data_preRT))

In [ ]:
set_determinism(seed=0)

training_data = data_preRT[:105]
validation_data = data_preRT[105:]

train_transforms = Compose(
     [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Resized(keys=["image", "label"], spatial_size=(128, 128, 64)),
        AsDiscreted(keys=["label"], to_onehot=3),
        ToTensord(keys=["image", "label"])
    ]
)

train_ds = Dataset(data=data_preRT, transform=train_transforms)
val_ds = Dataset(data=validation_data, transform=train_transforms)


train_dataloader = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_ds, batch_size=4, shuffle=False, num_workers=0)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import gc
import time
from tqdm import tqdm 

model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=3,
    channels=(16,32,64,128,256),
    strides=(2, 2, 2, 2),
).to(device)

loss_function = DiceLoss(softmax=True)
optimizer = torch.optim.Adam(model.parameters())
dice_metric = DiceMetric(include_background=True, reduction="mean")
max_epochs = 5

for epoch in range(max_epochs):
        torch.cuda.empty_cache()
        gc.collect()
        epoch_start = time.time()
        epoch_loss = []
        correct = 0
        total = 0
        print("-" * 10)
        print(f"epoch {epoch + 1}/{max_epochs}")
        model.train()
        
        training_losses = []
        for batch_data in tqdm(train_dataloader):
            images, labels = batch_data["image"].to(device), batch_data["label"].to(device)
            optimizer.zero_grad()
            outputs = model(images)
            #l = np.argmax(outputs[0], axis=0)
            #print(np.unique(l))
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss.append(loss.item())
            training_losses.append(loss.item())
            total += 1

            #print(f"train loss: {loss.item()}")
        
        #print(f"epoch {epoch + 1} average loss: {sum(epoch_loss)/total:.6f}")
        
        validation_losses = []
        with torch.no_grad():
            dice_scores = []
            for batch in tqdm(val_dataloader):
                images, labels = batch["image"].to(device), batch["label"].to(device)
                outputs = model(images)
                loss = loss_function(outputs, labels)
                #print(f"Validation loss: {loss}")
                validation_losses.append(loss.item())
                dice_metric(y_pred=outputs, y=labels)
            mean_dice = dice_metric.aggregate().item()
        print(f"Training mean loss: {np.mean(training_losses)}")
        print(f"Validation mean loss: {np.mean(validation_losses)}")
        print(f"Validation Mean Dice: {mean_dice:.6f}")





In [ ]:
image = val_ds[1]["image"]
label = val_ds[1]["label"]
image_with_batch = np.expand_dims(image, axis=0)
image_with_batch = torch.from_numpy(image_with_batch).float()
label_remove_one_hot = np.argmax(label, axis=0)

In [ ]:
image_vis = np.squeeze(image, axis=0)
plt.imshow(image_vis[:, :, 40], cmap='gray')

In [ ]:
image_with_batch = image_with_batch.to(device)


model.eval()
with torch.no_grad():  
    output = model(image_with_batch)
    output = output.cpu().numpy()
output_label_vis = np.squeeze(output, axis=0)
output_remove_one_hot = np.argmax(output_label_vis, axis=0)
print(np.unique(output_remove_one_hot))
positions = np.where(output_remove_one_hot == 1)
print(positions)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)  
plt.title("Ground Truth")
plt.imshow(label_remove_one_hot[:, :, 31], cmap='gray')
plt.axis('off') 

plt.subplot(2, 2, 2) 
plt.title("Model Output")
plt.imshow(output_remove_one_hot[:, :, 31], cmap='gray')
plt.axis('off') 

plt.tight_layout() 
plt.show()
